<a href="https://colab.research.google.com/github/nspll/nlp_final_project/blob/master/QA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pythainlp.tokenize import word_tokenize
import re

def search_answer_pattern (question , article):
  '''
  (str,str)=>list
  create pattern for finding answer 
  param:
    question: ประโยคคำถาม
    article : หัวเรื่องที่เลือกขึ้นมาจากคำถาม
  return string of pattern
  '''
  #from pythainlp.tokenize import word_tokenize
  #import re
  q_word = ['อะไร','ใคร','ที่ไหน','ไหน','ใด']
  
  
  
  q_list = word_tokenize(question.replace(article,'article'))
  pattern = ''
  map_ans = '([ก-ํ]+)'
  map_pattern = []
  for q in q_list:
    if q in q_word:
       map_pattern.append(map_ans)
    if q == 'article':
       map_pattern.append(article)
    elif q not in q_word :
       map_pattern.append(q)
                       
  
  
  return make_pattern(map_pattern,map_ans)


def make_pattern( map_pattern_list,map_ans):
  '''
  param:
  map_pattern_list : list of word to make pattern. ex, ['article','เป็น','เมืองหลวง','ของ','ประเทศ','(ก-ํ)'] 
  map_ans : format of answer's pattern . ex. (ก-ํ)
  จัดรูปแบบที่เป็นไปได้เกือบทั้งหมดของแพทเทิร์นนั้น
  return list of pattern strings 
  
  '''
 
  p_list=[]
  if map_pattern_list[0] == map_ans:
    map_pattern_list[0] = '\s'+str(map_ans)
  #elif  map_pattern_list[-1] == map_ans:
  map_pattern_list[-1] = str(map_ans)+str('\s')
 
  p_list.append(''.join(map_pattern_list)) 
  i = 1
  while i < len(map_pattern_list)-1:
    #เติมทีละ i ตัว จนถึง จน ช่องว่าง-1 ตัว
    p_pattern = '.*'.join(map_pattern_list[0:i+1])+''.join(map_pattern_list[i+1:]) 
    
    p_list.append( p_pattern) 
    i = i+1
  
 
  return p_list

def get_answer(question,article,article_dic):
  '''
  (str,str,dic) => str
  finding answer in article document
  #pattern search
  
  parameters:
   question : input question
   article  : name of selected article
   article_dic : dictionary where key = article , values = article file names
  return 
   answer that can find in this article
   
  '''
  file= open(article_dic[article], 'r',encoding ='utf-8')
  data = file.read()
 
  search_pattern = search_answer_pattern (question ,article)
  print (search_pattern)
  for pattern in search_pattern:
    ans = re.findall(pattern,data)
    if len(ans) != 0:
      return ans[0]
  return ''

def answer(question,freq_word_dic,article_list,article_dic):
  '''
  (str,dic,list,dic) => str
  finding answer in all selected documents until get answer or out of list
  parameters:
    question : input question
    freq_word_dic : dictionary where key = article , values = frequency of article in all files
    article_list : list of all articles
    article_dic : dictionary where key = article ,value = file of article 
    
  '''
  sorted_key_article_list = get_final_article_list(question,freq_word_dic,article_list,article_dic)
  for article in sorted_key_article_list:
    final_answer = get_answer(question,article,article_dic) 
    if final_answer != '':
      return final_answer
  return final_answer

def get_key_article_list (question ,article_list):
  '''
  (str,list) => list
  get key article list for searching answer
  paramerters:
   question : input question
   article_list : list of all articles
  return
   list of key article
   
  '''
  raw_tokens = word_tokenize(question)
  key_article_list = []
  i = 0
  pivot_s = 0
  pivot_e = 0
  while i < len(raw_tokens)-1:
    j = i
    pivot_s = i
    pivot_e = 0
    while j < len(raw_tokens):
        tmp = ''.join(raw_tokens[i:j+1])
        if tmp in article_list:
          #print (tmp)
          pivot_s  = i
          pivot_e = j
        
        j = j+1
    if pivot_e != 0:
      i = pivot_e
      prop_article = ''.join(raw_tokens[pivot_s:pivot_e+1])
    i = i+1
  return key_article_list

def prioritize_keyword ( key_article_list , freq_word_dic):
  '''
  (list,dic)=>list
  จัดลำดับความสำคัญของชื่อหัวข้อ
  parameters :
  key_article_list : ชื่อหัวข้อที่มีอยู่ในคำถาม
  freq_word_doc : เก็บ key เป็น ชื่อหัวข้อ (article) ค่าเป็นจำนวนความถี่ โดยเรียงความถี่มาอยู่แล้วจากมากไปน้อย
  return prioritize article list
  
  '''
  #for i in key_article_list:
   #print (i,freq_word_dic[i])
  ans =   sorted ( [ (freq_word_dic[article],article) for article in key_article_list if freq_word_dic[article] ] )
  article_sorted_list = [item[1]for item in ans]
  
  return article_sorted_list

def get_article_score ( data, key_article_list ):
  '''
  (str,str) => int 
  ถ้ามีคำใน key_article เยอะก็น่าจะมีความน่าจะเป็นที่จะมีคำตอบอยู่ในไฟล์นี้
  param :
    question : คำถามทั้งหมดที่ถาม
    article_file_name : ชื่อไฟล์ของหัวข้อนั้นๆ
   
  '''
  score = 0 
  file = open( article_file_name ,'r')
  data = file.read()
  for q in key_article_list :
    if q in data:
      score+=1
  return score

